In [ ]:
import os
import sys

In [2]:
import ase.db
import numpy as np
import torch
from pymatgen.io.ase import AseAtomsAdaptor

In [3]:
test_db = ase.db.connect("../data/test_structs_1k.db")

In [5]:
test_sys = test_db.get_atoms(5)

In [6]:
def time_get_neighbor_list(atoms_db, radius=6):
    for row in atoms_db.select():
        atoms = row.toatoms()
        structure = AseAtomsAdaptor.get_structure(atoms)
        
        neighbors = structure.get_neighbor_list(r=radius)

In [7]:
def time_get_all_neighbors(atoms_db, radius=6):
    for row in atoms_db.select():
        atoms = row.toatoms()
        structure = AseAtomsAdaptor.get_structure(atoms)
        
        neighbors = structure.get_all_neighbors(r=radius, include_index=True)

In [8]:
test_structure = AseAtomsAdaptor.get_structure(test_sys)

In [9]:
%timeit nb1 = test_structure.get_neighbor_list(r=6.0)

893 µs ± 25 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%timeit nb2 = test_structure.get_all_neighbors(r=6.0, include_index=True)

2.33 ms ± 43.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%timeit test_sys.get_all_distances(mic=True)

1.75 ms ± 19 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
%timeit nb1_fun = time_get_neighbor_list(test_db)

2.92 s ± 57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit nb2_fun = time_get_all_neighbors(test_db)

9.37 s ± 135 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# appears the get_neighbor_list method is at least 3x faster